# Optimizing Pipelines Project

## Objective
This project is part of the Udacity Azure ML Nanodegree.  
In this project, we build and optimize an Azure ML pipeline using the Python SDK and a provided Scikit-learn model.  
This model is then compared to an Azure AutoML run.  

## 3. AutoML run (version 2)

This version of the AutoML run was tested, since the first version was producing lower *accuracy* values than the othes obtained with the hyperparameter run.  

The hypothesis for the probable cause is that both approaches were not working under equal basis.  
Although both approaches use a train-test split of the dataset, the AutoML is only using the train dataset and n-cross validation (as requested in the initial code provided for the project).  

To get a fair comparison between both approaches, these must use the same train-test datasets, so the n-cross validation parameter should not be stated in AutoML to allow the use of the test dataset.  

Additionally, the ensemble models were disabled, to get a fair comparison vs. a single algorithmn and not a combination of them.  

### Initialize workspace and experiment

In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspac<<e.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126032
Azure region: southcentralus
Subscription id: ebee3a56-4c54-406a-b732-174015826780
Resource group: aml-quickstarts-126032


### Create Cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

#- Define cluster name
cluster_name = "compute-cluster"

#- Define cluster configuration
cluster_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                       max_nodes=4)

#- Create cluster
compute_target = ComputeTarget.create(ws, cluster_name, cluster_config)

#- Set timeout
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

#- Get cluster status
print(compute_target.get_status().serialize())

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-11-13T23:28:56.779000+00:00', 'errors': None, 'creationTime': '2020-11-13T23:28:53.549617+00:00', 'modifiedTime': '2020-11-13T23:29:09.205539+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


### Load dataset

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

#- This library was included to load the dataset
from azureml.core import Dataset

#- Define path to data
web_path ='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

#- Create tabular dataset
ds_ml = Dataset.Tabular.from_delimited_files(path=web_path)

### Clean dataset

In [4]:
from train import clean_data

# Use the clean_data function to clean your data.
# x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x_ml, y_ml = clean_data(ds_ml)

### Create training and testing datasets

In [5]:
from sklearn.model_selection import train_test_split

#- Split into 70-30 proportion, since is the general recommended value in the field
#- Set random_state to 0, to ensure that the same random combination is used between runs
x_ml_train, x_ml_val, y_ml_train, y_ml_val = train_test_split(x_ml, y_ml, test_size=0.30, random_state=0)

In [6]:
import pandas as pd

#- Concatenate input dataset and label dataset
training_data = pd.concat ([x_ml_train, y_ml_train], axis=1)
validation_data = pd.concat ([x_ml_val, y_ml_val], axis=1)

In [7]:
#- Create folder to save train data as csv
if not os.path.isdir('data'):
    os.mkdir('data')
    
#- Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(training_data).to_csv("data/train_data.csv", index=False)
pd.DataFrame(validation_data).to_csv("data/val_data.csv", index=False)

ds2 = ws.get_default_datastore()
ds2.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)


#- Upload the training and validation data as a tabular dataset for access during training on remote compute
training_dataset = Dataset.Tabular.from_delimited_files(path=ds2.path('bankmarketing/train_data.csv'))
validation_dataset = Dataset.Tabular.from_delimited_files(path=ds2.path('bankmarketing/val_data.csv'))

Uploading an estimated of 2 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 2
Uploading ./data/val_data.csv
Uploaded ./data/val_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


### Set AutoML run configuration

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric='accuracy',
    training_data=training_dataset,
    label_column_name="y",
    #n_cross_validations=2,
        
    debug_log = 'automl_errors.log',
    compute_target=compute_target,
    validation_data = validation_dataset,
    enable_voting_ensemble=False,
    enable_stack_ensemble=False,
    enable_onnx_compatible_models=True
)

### Submit AutoML run

In [9]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output = False)

Running on remote.


In [10]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_127b9952-d7d0-4910-8560-302e9a40400f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
#- Wait for the remote run to complete
remote_run.wait_for_completion()

{'runId': 'AutoML_127b9952-d7d0-4910-8560-302e9a40400f',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-11-13T23:30:43.915862Z',
 'endTimeUtc': '2020-11-14T00:11:48.86681Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"ebee3a56-4c54-406a-b732-174015826780","resource_group":"aml-quickstarts-126032","workspace_name":"quick-starts-ws-126032","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"

### Show AutoML run results

In [12]:
#- Show results
from azureml.widgets import RunDetails

RunDetails(remote_run).show() 

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Images of the AutoML runs

![AutoML results 01](images/A_03.png)
![AutoML results 02](images/A_04.png)
![AutoML results 03](images/AutoML_04.png)

### Find and save best model

In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
#- Retrieve model explanation
from azureml.core.run import Run

model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=exp, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

#- Get the best run object
best_run_ml, fitted_model_ml = remote_run.get_output()

#- Print the properties of the model
print (best_run_ml)

AutoML_127b9952-d7d0-4910-8560-302e9a40400f_ModelExplain
Run(Experiment: udacity-project,
Id: AutoML_127b9952-d7d0-4910-8560-302e9a40400f_1,
Type: azureml.scriptrun,
Status: Completed)


In [14]:
print (fitted_model_ml)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('MaxAbsScaler', MaxAbsScaler(copy...
                                   colsample_bylevel=1, colsample_bynode=1,
                                   colsample_bytree=1, gamma=0,
                                   learning_rate=0.1, max_delta_step=0,
                                   max_depth=3, min_child_weight=1, missing=nan,
                                   n_estimators=100, n_jobs=1, nthread=None,
                     

### Images of Model Explanation

![AutoML results 03](images/AutoML_05.png)
![AutoML results 04](images/AutoML_06.png)
![AutoML results 05](images/AutoML_07.png)

In [15]:
#- Retrieve best ONNX model
best_run_onnx, onnx_mdl = remote_run.get_output(return_onnx_model=True)

In [16]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

#- Save best ONNX model
onnx_fl_path = "./best_model_a2.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

### Cluster clean up

In [17]:
#- Delete compute target
compute_target.delete()